In [0]:
WITH state_emissions AS (
    SELECT
        state_abbr,
        SUM(
            CAST(
                REPLACE(
                    `GHG emissions mtons CO2e`,
                    ',',
                    ''
                ) AS DOUBLE
            )
        ) AS total_emissions
    FROM emissions_data_2023
    GROUP BY state_abbr
),
top_10_states AS (
    SELECT
        state_abbr,
        total_emissions
    FROM state_emissions
    ORDER BY total_emissions DESC
    LIMIT 10
),
country_total AS (
    SELECT
        SUM(total_emissions) AS country_emissions
    FROM state_emissions
),
top_10_with_percent AS (
    SELECT
        t.state_abbr,
        t.total_emissions,
        ROUND(
            100 * t.total_emissions / c.country_emissions,
            2
        ) AS percent_of_country
    FROM top_10_states t
    CROSS JOIN country_total c
)
SELECT
    SUM(percent_of_country) AS total_percent
FROM top_10_with_percent